### Import lib

In [1]:
import sys
sys.path.append('..')

In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm

import config
from models.coordinate import Coordinate

### Read data

In [3]:
def read_data(data_path):
    list_cities = [None]

    with open(data_path, 'r') as f:
        data = f.readlines()
        NUMBER_CITIES = int(data[3].split(':')[-1].strip())

        for line in data[6: 6 + NUMBER_CITIES]:
            _, x, y = list(map(int, line.split()))
            list_cities.append(Coordinate(x, y))

    return list_cities, NUMBER_CITIES

list_cities = [None]
NUMBER_CITIES = [None]
NUMBER_FACTOR = len(config.DATA_PATH)

for path in config.DATA_PATH:    
    factorial_list_cities, factorial_NUMBER_CITIES = read_data(path)
    list_cities.append(factorial_list_cities)
    NUMBER_CITIES.append(factorial_NUMBER_CITIES)

print(list_cities, NUMBER_CITIES)

[None, [None, Coordinate(x=37, y=52), Coordinate(x=49, y=49), Coordinate(x=52, y=64), Coordinate(x=20, y=26), Coordinate(x=40, y=30), Coordinate(x=21, y=47), Coordinate(x=17, y=63), Coordinate(x=31, y=62), Coordinate(x=52, y=33), Coordinate(x=51, y=21)], [None, Coordinate(x=37, y=52), Coordinate(x=49, y=49), Coordinate(x=52, y=64), Coordinate(x=20, y=26), Coordinate(x=40, y=30), Coordinate(x=21, y=47), Coordinate(x=17, y=63), Coordinate(x=31, y=62), Coordinate(x=52, y=33), Coordinate(x=51, y=21), Coordinate(x=42, y=41), Coordinate(x=31, y=32), Coordinate(x=5, y=25), Coordinate(x=12, y=42), Coordinate(x=36, y=16), Coordinate(x=52, y=41), Coordinate(x=27, y=23), Coordinate(x=17, y=33), Coordinate(x=13, y=13), Coordinate(x=57, y=58), Coordinate(x=62, y=42), Coordinate(x=42, y=57), Coordinate(x=16, y=57), Coordinate(x=8, y=52), Coordinate(x=7, y=38), Coordinate(x=27, y=68), Coordinate(x=30, y=48), Coordinate(x=43, y=67), Coordinate(x=58, y=48), Coordinate(x=58, y=27), Coordinate(x=37, y=69

### Initialize Population

In [4]:
def init_population(factor_index):
    population = [None]

    for _ in range(1, config.POPULATION_SIZE + 1):
        while True:
            individual = random.sample(
                range(1, NUMBER_CITIES[factor_index] + 1), NUMBER_CITIES[factor_index])

            assert len(set(individual)) == NUMBER_CITIES[factor_index]
            if individual not in population:
                population.append(individual)
                break
        
    return population

index_max = np.argmax([0] + NUMBER_CITIES[1:])
population = init_population(index_max)


In [5]:
def decoded(individual, factor_index):
    return [element for element in individual if element <= NUMBER_CITIES[factor_index]]

### Adaptive function

In [6]:
def euclidean_distance(index_city1, index_city2, factor_index):
    tmp_list_cities = list_cities[factor_index]

    return np.linalg.norm((tmp_list_cities[index_city1].x - tmp_list_cities[index_city2].x,
                           tmp_list_cities[index_city1].y - tmp_list_cities[index_city2].y))


DISTANCE_MATRIX = [None]

for factor_index in range(1, NUMBER_FACTOR + 1):
    tmp_LENGTH_MATRIX = NUMBER_CITIES[factor_index] + 1
    tmp_DISTANCE_MATRIX = np.zeros((tmp_LENGTH_MATRIX, tmp_LENGTH_MATRIX))

    for i in range(1, NUMBER_CITIES[factor_index] + 1):
        for j in range(1, NUMBER_CITIES[factor_index] + 1):
            if i < j:
                tmp_DISTANCE_MATRIX[i][j] = euclidean_distance(
                    i, j, factor_index)
            elif i > j:
                tmp_DISTANCE_MATRIX[i][j] = tmp_DISTANCE_MATRIX[j][i]
            else:
                tmp_DISTANCE_MATRIX[i][j] = 0

    DISTANCE_MATRIX.append(tmp_DISTANCE_MATRIX)


In [8]:
def cost(individual, factor_index):
    c = 0
    for i in range(len(individual)):
        c += DISTANCE_MATRIX[factor_index][individual[i]][individual[i+1] if i != len(individual) - 1
                                            else individual[0]]

    return c


### Ranking

In [9]:
ranks = [None]

for factor_index in range(1, NUMBER_FACTOR + 1):
    list_cost = [cost(decoded(individual, factor_index), factor_index) for individual in population[1:]]
    ranks.append([None] + [it + 1 for it in np.argsort(list_cost)])

### Skill-factor & Scalar-fitness

In [10]:
skill_factor = [None]
scalar_fitness = [None]

for individual_index in range(1, config.POPULATION_SIZE + 1): 
    tmp_rank = [None]
    for rank in ranks[1:]:
        tmp_rank.append(rank[individual_index])

    skill_factor.append(np.argmin(tmp_rank[1:]) + 1)
    scalar_fitness.append(1/min(tmp_rank[1:]))

### Select parents

In [11]:
def select_parents(population):
    candidate_indexs = random.sample(range(1, config.POPULATION_SIZE + 1), 4)
    winner_indexs = []
    winner_indexs.append(candidate_indexs[0] if scalar_fitness[candidate_indexs[0]] < scalar_fitness[candidate_indexs[1]] else candidate_indexs[1])
    winner_indexs.append(candidate_indexs[2] if scalar_fitness[candidate_indexs[2]] < scalar_fitness[candidate_indexs[3]] else candidate_indexs[3])
    return ((population[winner_indexs[0]], population[winner_indexs[1]]), (skill_factor[winner_indexs[0]], skill_factor[winner_indexs[1]]))


def select_cut_points(parent):
    pos1, pos2 = tuple(random.sample(range(len(parent)), 2))

    return (pos1, pos2) if pos1 > pos2 else (pos2, pos1)


### Crossover

In [13]:
def exec_cross(parent1, parent2, pos1, pos2):
    child = parent1[pos1: pos2 + 1]
    
    tmp = []
    for i in range(pos2 + 1, pos2 + 1 + len(parent2)):
        if i >= len(parent2):
            i %= len(parent2)
        if parent2[i] not in child:
            tmp.append(parent2[i])

    return (tmp[-pos1:] if pos1 != 0 else []) + child + tmp[:len(tmp) - pos1]


def crossover(parents, parent_skill_factors):
    if parent_skill_factors[0] == parent_skill_factors[1] or random.random() < config.RMP:
        pos1, pos2 = select_cut_points(parents[0])

        children = []
        children.append(exec_cross(parents[0], parents[1], pos1, pos2))
        children.append(exec_cross(parents[1], parents[0], pos1, pos2))
        return (tuple(children), tuple(parent_skill_factors))

    return None, None

def circle_crossover(parents):
    pass
    

### Mutation

In [14]:
def mutation(parents, parent_skill_factors):
    if parent_skill_factors[0] != parent_skill_factors[1] or random.random() >= config.RMP:
        children = []
        children_skill_factors = []

        for parent, parent_skill_factor in zip(parents, parent_skill_factors):
            pos1, pos2 = select_cut_points(parent)

            parent[pos1], parent[pos2] = parent[pos2], parent[pos1]
            children.append(parent)
            children_skill_factors.append(parent_skill_factor)

        return (tuple(children), tuple(children_skill_factors))

    return None, None


### Generate offspring

In [15]:
def gen_offspring(population):
    offspring = [None]
    offspring_skill_factor = [None]

    while len(offspring) < len(population):
        parents, parent_skill_factors = select_parents(population)

        children_cross, children_cross_skill_factors = crossover(parents, parent_skill_factors)
        if children_cross != None:
            for child, child_skill_factor in zip(children_cross, children_cross_skill_factors):
                offspring.append(child)
                offspring_skill_factor.append(child_skill_factor)

        children_mut, children_mut_skill_factors = mutation(parents, parent_skill_factors)
        if children_mut != None:
            for child, child_skill_factor in zip(children_mut, children_mut_skill_factors):
                offspring.append(child)
                offspring_skill_factor.append(child_skill_factor)
                
    # selective
    assert len(offspring) >= len(population)
    

    return offspring


In [16]:
gen_offspring(population)

[None,
 [46,
  37,
  36,
  26,
  2,
  43,
  15,
  34,
  11,
  44,
  30,
  23,
  3,
  45,
  28,
  40,
  18,
  9,
  14,
  47,
  25,
  50,
  21,
  5,
  17,
  35,
  10,
  22,
  49,
  7,
  8,
  48,
  12,
  27,
  13,
  51,
  16,
  24,
  32,
  6,
  31,
  20,
  33,
  42,
  19,
  38,
  39,
  1,
  4,
  41,
  29],
 [12,
  17,
  24,
  4,
  15,
  5,
  1,
  46,
  11,
  36,
  13,
  47,
  29,
  14,
  7,
  34,
  38,
  6,
  22,
  37,
  2,
  20,
  31,
  44,
  49,
  21,
  33,
  10,
  25,
  48,
  27,
  26,
  30,
  43,
  3,
  23,
  9,
  40,
  19,
  16,
  28,
  18,
  41,
  39,
  51,
  35,
  42,
  50,
  8,
  45,
  32],
 [25,
  48,
  27,
  26,
  30,
  43,
  3,
  23,
  9,
  40,
  19,
  16,
  28,
  18,
  24,
  39,
  51,
  35,
  42,
  50,
  8,
  45,
  32,
  12,
  17,
  41,
  4,
  15,
  5,
  1,
  46,
  11,
  36,
  13,
  47,
  29,
  14,
  7,
  34,
  38,
  6,
  22,
  37,
  2,
  20,
  31,
  44,
  49,
  21,
  33,
  10],
 [49,
  7,
  8,
  48,
  12,
  27,
  13,
  51,
  16,
  24,
  32,
  6,
  31,
  20,
  33,
  42,
  19,


In [17]:
# history = [None]

# for i in tqdm(range(config.NUM_GENERATIONS)):
#     offspring, best_cost = gen_offspring(population)
#     history.append(best_cost)
#     population = offspring